In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import csv

Lecture des données

In [2]:
df_match= pd.read_csv('train_with_1000_join_tours_no_null.csv', low_memory=False)

In [3]:
df_match.head()

,ID_T,ID_C_T,RANK_T,ID1_G,ID2_G,ID_T_G,ID_R_G,winner,POINT_P1,POS_P1,POINT_P2,POS_P2,diff_rating
0,8889,2,2,935,838,8889,4,1,745.642857,74.523810,613.333333,90.742857,-16.219048
1,8886,1,6,21497,20655,8886,4,2,101.333333,452.368421,161.241176,350.505882,101.862539
2,8957,3,1,11373,1264,8957,5,2,510.990476,114.242857,277.290476,220.085714,-105.842857
3,8891,3,2,15323,23433,8891,9,1,0.000000,1000.000000,0.000000,1000.000000,0.000000
4,8898,1,2,6208,1049,8898,1,1,193.662500,289.425000,19.000000,848.666667,-559.241667


In [4]:
df_stats_tours = pd.DataFrame()
df_stats_tours['j1_win']= df_match[df_match['winner'] == 1].groupby(['ID1_G', 'RANK_T'])['winner'].count()
df_stats_tours['j1_loose']= df_match[df_match['winner'] == 2].groupby(['ID1_G', 'RANK_T'])['winner'].count()
df_stats_tours['j2_win']= df_match[df_match['winner'] == 1].groupby(['ID2_G', 'RANK_T'])['winner'].count()
df_stats_tours['j2_loose']= df_match[df_match['winner'] == 2].groupby(['ID2_G', 'RANK_T'])['winner'].count()
df_stats_tours=df_stats_tours.fillna(0)
df_stats_tours['j_win']= df_stats_tours.j1_win+ df_stats_tours.j2_win
df_stats_tours['j_loose']= df_stats_tours.j2_loose+ df_stats_tours.j2_loose
df_stats_tours['j_total']= df_stats_tours.j_win+ df_stats_tours.j_loose
df_stats_tours['ratio_tours']= df_stats_tours.j_win / df_stats_tours.j_total
df_stats_tours = df_stats_tours.drop(['j1_win', 'j2_win', 'j1_loose', 'j2_loose','j_loose', 'j_total'], 1)
df_stats_tours = df_stats_tours.reset_index()
df_stats_tours.head(10)
# df_tmp = df_stats_tours[['ID1_G', 'j_win']]
# df_tmp.head(10)

,ID1_G,RANK_T,j_win,ratio_tours
0,7,2,30.0,0.441176
1,7,3,10.0,0.500000
2,7,4,14.0,0.500000
3,7,5,5.0,0.714286
4,7,6,4.0,0.285714
5,10,2,5.0,1.000000
6,10,3,2.0,1.000000
7,10,4,2.0,1.000000
8,10,5,1.0,1.000000
9,12,1,7.0,0.538462


In [5]:
df_join_stats_tours= df_match.merge(df_stats_tours, on=['ID1_G', 'RANK_T'], how='left')
df_join_stats_tours.rename(columns={'j_win': 'j1_win_ontour'}, inplace=True)
df_join_stats_tours.rename(columns={'ratio_tours': 'j1_win_ontour_ratio'}, inplace=True)
df_join_stats_tours= df_join_stats_tours.merge(df_stats_tours, left_on=['ID2_G', 'RANK_T'], right_on=['ID1_G', 'RANK_T'], how='left')
df_join_stats_tours.rename(columns={'j_win': 'j2_win_ontour'}, inplace=True)
df_join_stats_tours.rename(columns={'ratio_tours': 'j2_win_ontour_ratio'}, inplace=True)
df_join_stats_tours = df_join_stats_tours.drop('ID1_G_y', 1)
df_join_stats_tours.rename(columns={'ID1_G_x': 'ID1_G'}, inplace=True)
df_join_stats_tours=df_join_stats_tours.fillna(0)
df_join_stats_tours=df_join_stats_tours.drop('ID_T', 1)
df_join_stats_tours['diff_wins_for_tour']= df_join_stats_tours.j1_win_ontour - df_join_stats_tours.j2_win_ontour
df_join_stats_tours['diff_wins_for_tour_ratio']= df_join_stats_tours.j1_win_ontour_ratio - df_join_stats_tours.j2_win_ontour_ratio
df_join_stats_tours=df_join_stats_tours.drop(['j1_win_ontour', 'j2_win_ontour','j1_win_ontour_ratio' , 'j2_win_ontour_ratio'] , 1)
df_train_tours_stats = df_join_stats_tours[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_R_G', 'POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'winner']]
# df_train_tours_stats = df_join_stats_tours[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_R_G', 'ID_C_T', 'POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'winner']]
# df_train_tours_stats.to_csv('train_tours_stats.csv', sep=',', index=False)
df_train_tours_stats.head(10)

,ID1_G,ID2_G,ID_T_G,RANK_T,ID_R_G,POINT_P1,POS_P1,POINT_P2,POS_P2,diff_rating,diff_wins_for_tour,diff_wins_for_tour_ratio,winner
0,935,838,8889,2,4,745.642857,74.523810,613.333333,90.742857,-16.219048,7.0,0.042156,1
1,21497,20655,8886,6,4,101.333333,452.368421,161.241176,350.505882,101.862539,1.0,-0.200000,2
2,11373,1264,8957,1,5,510.990476,114.242857,277.290476,220.085714,-105.842857,12.0,0.030094,2
3,15323,23433,8891,2,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,28.0,-0.605263,1
4,6208,1049,8898,2,1,193.662500,289.425000,19.000000,848.666667,-559.241667,-5.0,-0.389655,1
5,22609,25447,8899,3,4,0.000000,1000.000000,0.000000,1000.000000,0.000000,-2.0,-0.500000,2
6,3468,6431,9091,1,4,0.000000,1000.000000,454.904762,155.333333,844.666667,-56.0,-0.518519,2
7,22347,25700,8903,3,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,13.0,-0.045455,1
8,17020,4178,9085,1,5,138.546053,341.342105,271.542484,276.352941,64.989164,-2.0,0.007590,2
9,25813,21659,8906,2,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,-1.0,0.500000,2


In [6]:
# cols= df_join_stats_tours.columns.tolist()
# # cols= cols[:7]+cols[8:]+[cols[7]]
# df_join_stats_tours= df_join_stats_tours[cols]
# #df_join_stats_tours

In [7]:
X= df_train_tours_stats.values
# print X[:1,0]
# X= df_join_stats_tours.values
# print X

In [8]:
clf = RandomForestClassifier()
clf.fit(X[:60000,:12], X[:60000,12])  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
print clf.score(X[:60000,:12], X[:60000,12])
print clf.score(X[60000:,:12], X[60000:,12])

0.983733333333
0.692505615683


In [10]:
print X[1,:12]

[  2.14970000e+04   2.06550000e+04   8.88600000e+03   6.00000000e+00
   4.00000000e+00   1.01333333e+02   4.52368421e+02   1.61241176e+02
   3.50505882e+02   1.01862539e+02   1.00000000e+00  -2.00000000e-01]


In [11]:
test_public= pd.read_csv('test_public_tours.csv', low_memory=False)
test_public.head()

,ID1_G,ID2_G,ID_T_G,ID_R_G,POINT_P1,POS_P1,POINT_P2,POS_P2,diff_rating,RANK_T
0,28073,36278,13296,5,112.962963,490.274074,0.000000,1000.000000,-509.725926,1
1,10593,27496,13440,4,63.140351,541.245614,32.228571,740.885714,-199.640100,0
2,55090,55049,13303,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,1
3,31186,1170,13468,4,0.000000,1000.000000,26.987342,746.506329,253.493671,0
4,22235,9658,13308,3,58.898204,572.682635,168.885714,283.980952,288.701682,1


In [12]:
df_join_stats_tours= test_public.merge(df_stats_tours, on=['ID1_G', 'RANK_T'], how='left')
df_join_stats_tours.rename(columns={'j_win': 'j1_win_ontour'}, inplace=True)
df_join_stats_tours.rename(columns={'ratio_tours': 'j1_win_ontour_ratio'}, inplace=True)
df_join_stats_tours= df_join_stats_tours.merge(df_stats_tours, left_on=['ID2_G', 'RANK_T'], right_on=['ID1_G', 'RANK_T'], how='left')
df_join_stats_tours.rename(columns={'j_win': 'j2_win_ontour'}, inplace=True)
df_join_stats_tours.rename(columns={'ratio_tours': 'j2_win_ontour_ratio'}, inplace=True)
df_join_stats_tours = df_join_stats_tours.drop('ID1_G_y', 1)
df_join_stats_tours.rename(columns={'ID1_G_x': 'ID1_G'}, inplace=True)
df_join_stats_tours=df_join_stats_tours.fillna(0)
df_join_stats_tours['diff_wins_for_tour']= df_join_stats_tours.j1_win_ontour - df_join_stats_tours.j2_win_ontour
df_join_stats_tours['diff_wins_for_tour_ratio']= df_join_stats_tours.j1_win_ontour_ratio - df_join_stats_tours.j2_win_ontour_ratio
df_join_stats_tours=df_join_stats_tours.drop(['j1_win_ontour', 'j2_win_ontour','j1_win_ontour_ratio' , 'j2_win_ontour_ratio'] , 1)
df_test_tours_stats = df_join_stats_tours[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_R_G','POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio']]
df_test_tours_stats.to_csv('test_tours_stats.csv', sep=',', index=False)
df_test_tours_stats.head(10)

,ID1_G,ID2_G,ID_T_G,RANK_T,ID_R_G,POINT_P1,POS_P1,POINT_P2,POS_P2,diff_rating,diff_wins_for_tour,diff_wins_for_tour_ratio
0,28073,36278,13296,1,5,112.962963,490.274074,0.000000,1000.000000,-509.725926,34.0,0.531250
1,10593,27496,13440,0,4,63.140351,541.245614,32.228571,740.885714,-199.640100,4.0,-0.429268
2,55090,55049,13303,1,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,0.0,0.000000
3,31186,1170,13468,0,4,0.000000,1000.000000,26.987342,746.506329,253.493671,-3.0,0.214286
4,22235,9658,13308,1,3,58.898204,572.682635,168.885714,283.980952,288.701682,-60.0,-0.147583
5,1837,762,12835,2,9,1221.309524,51.990476,769.490476,64.495238,-12.504762,45.0,0.111943
6,26144,14527,13489,0,5,0.000000,1000.000000,26.981818,739.309091,260.690909,-3.0,0.078947
7,18122,6276,13317,1,3,64.131148,541.185792,117.056911,405.731707,135.454085,35.0,-0.028926
8,19946,10505,13500,0,9,45.118421,660.302632,154.370968,309.107527,351.195105,12.0,0.375000
9,2034,10901,13321,1,4,72.722222,510.633333,152.024272,376.504854,134.128479,7.0,0.075077


In [13]:
# df_test_public_joined = test_public.merge(df_stats_tours, on=['ID1_G', 'RANK_T'], how='left')
# df_test_public_joined = test_public.merge(df_stats_tours, left_on=['ID2_G', 'RANK_T'], right_on=['ID1_G', 'RANK_T'], how='left')
# df_test_public_joined = df_test_public_joined.drop('ID1_G_y', 1)
# df_test_public_joined.rename(columns={'ID1_G_x': 'ID1_G'}, inplace=True)
# df_test_public_joined=df_test_public_joined.fillna(0)
# df_test_public_joined.head(10)

In [14]:
# df_test_public_joined.to_csv('tochangeorder.csv', sep=';')

In [ ]:
X_test= df_test_tours_stats.values

In [ ]:
pred  = clf.predict(X_test)
print pred

In [ ]:
df_leaderboard_tours_stats = df_test_tours_stats[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_R_G','POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio']]
df_leaderboard_tours_stats['predictions'] = pred
df_leaderboard_tours_stats = df_leaderboard_tours_stats.drop(['RANK_T', 'POINT_P1', 'POINT_P2', 'POS_P1', 'POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio'], 1)
df_leaderboard_tours_stats.head(10)

In [ ]:
df_leaderboard_tours_stats.to_csv('leaderboard_tours_stats.csv', sep=',', index=False, header= False)